1. The following file exists in the Databricks file system:
/FileStore/sample/sandp_stocks.csv
You can view a version of the file here:
https://gitlab.com/opstar/share20/-/raw/master/sandp_stocks.csv


In [ ]:
from pyspark.sql.functions import sum,avg

2. Use the spark.read method to create a dataframe from the file.

Each tuple consists of daily market prices and volumes for stocks for a 5 year period.
The stock name is identified by the letters in the column on the right (e.g. AAL is
American Airlines).

The dataframe should look like this:

In [ ]:
+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
+----------+-----+-----+-----+-----+--------+----+

In [ ]:
spr = spark.read\
 .format('csv')\
 .option('header', 'true')\
 .option('inferSchema','false')\
 .load('/FileStore/tables/sandp_stocks.csv')

3. Convert the columns to some appropriate data types.

In [ ]:
spr = spr.selectExpr( \
                    'date(date)'\
                   ,'float(open)'\
                   ,'float(high)'\
                   ,'float(low)'\
                   ,'float(close)'\
                   ,'int(volume)'
                   ,'name'
                   ,'year(date) AS year')

spr.show()

4. Use groupBy to aggregate the data and calculate the total volume for each stock (name). The finished result should look like this:

In [ ]:
+----+-----------+
|name|sum(volume)|
+----+-----------+
|ALXN| 2218905439|
| AIV| 1509190118|
| AVY|  957891856|

In [ ]:
# 4. Use groupBy to aggregate the data and calculate the total volume for each stock
f = spr.groupBy('name').agg(sum('volume'))
 
f.show()

5. Now calculate the average closing stock price for Microsoft (MSFT) for each year.
The finished result should look like this:
avclose.show()

In [ ]:
+----+--------+
|year|avgclose|
+----+--------+
|2013| 33.0902|
|2014| 42.4533|
|2015|46.71380|
|2018| 90.2523|

In [ ]:
g = spr.where(spr.name=='MSFT').groupBy('year').agg(
    avg('close').alias('avgclose')).orderBy('year')
 
g.show()



# Alternative version reading the year from the date string
g = spr.where(spr.name=='MSFT')\
    .groupBy(spr.date[0:4].alias('year')).agg(
        avg('close').alias('avgclose')).orderBy('year')
 
g.show()



# Example of the alternative dictionary syntax for the aggregate
#  -- no need to import the function but you can't so easily alias the aggregated column
g = spr.where(spr.name=='MSFT')\
    .groupBy('year')\
    .agg({'close':'avg','open':'avg'})\
    .orderBy('year')
 
g.show()

from pyspark.sql.functions import col
g.orderBy(g.year.desc(), g.avgclose.asc()).show()
 
g.orderBy(col('year').desc(), col('avgclose').asc()).show()